In [52]:
from pystac_client import Client
import pandas as pd
import os, requests, rasterio

CATALOG_URL = "https://earth-search.aws.element84.com/v1"

BAND_MAP = {
    "B01": "coastal",
    "B02": "blue",
    "B03": "green",
    "B04": "red",
    "B05": "rededge1",
    "B06": "rededge2",
    "B07": "rededge3",
    "B08": "nir",
    "B8A": "nir08",
    "B09": "nir09",
    "B11": "swir16",
    "B12": "swir22",
    "SCL": "scl"
}

def search_s2_l2a(aoi, start_date, end_date, cloud_cover=80):
    catalog = Client.open(CATALOG_URL)
    search = catalog.search(collections=["sentinel-2-l2a"],
                            intersects=aoi,
                            datetime=f"{start_date}/{end_date}",
                            query={"eo:cloud_cover": {"lt": cloud_cover}})
    return list(search.get_items())

def summarize_metadata(items):
    rows = []
    for it in items:
        p = it.properties
        rows.append({
            "id": it.id,
            "datetime": p.get("datetime"),
            "cloud_cover": p.get("eo:cloud_cover"),
            "sun_azimuth": p.get("view:sun_azimuth"),
            "sun_elevation": p.get("view:sun_elevation"),
            "asset_count": len(it.assets)
        })
    return pd.DataFrame(rows)

def download_band(item, band_name, out_dir):
    key = BAND_MAP.get(band_name, band_name)
    if key not in item.assets:
        return None
    subdir = os.path.join(out_dir, item.id)
    os.makedirs(subdir, exist_ok=True)
    out_file = os.path.join(subdir, f"{band_name}.tif")
    if os.path.exists(out_file):
        return out_file
    url = item.assets[key].href
    r = requests.get(url, stream=True)
    r.raise_for_status()
    with open(out_file, "wb") as f:
        for chunk in r.iter_content(8192):
            f.write(chunk)
    return out_file

def download_all_bands(item, out_dir, bands, scl_band):
    for b in bands + scl_band:
        download_band(item, b, out_dir)

def merge_bands_to_multitif(scene_dir, out_path, bands, scl_band):
    paths = [os.path.join(scene_dir, f"{b}.tif") for b in bands + scl_band]
    paths = [p for p in paths if os.path.exists(p)]
    if not paths:
        return None
    ds_list = [rasterio.open(p) for p in paths]
    meta = ds_list[0].meta.copy()
    meta.update(count=len(ds_list))
    os.makedirs(os.path.dirname(out_path), exist_ok=True)
    with rasterio.open(out_path, "w", **meta) as dst:
        for i, src in enumerate(ds_list, 1):
            dst.write(src.read(1), i)
    for ds in ds_list:
        ds.close()
    return out_path

# Download Study Site
# ~ --------------------- ~ ##

In [54]:
# YF
from datetime import datetime
import geopandas as gpd

CONFIG_PATH = r"D:\planetscope_lake_ice\sentinel_config.yaml"

BANDS = ["B01","B02","B03","B04","B05","B06","B07","B08","B8A","B09","B11","B12"]
SCL_BAND = ["SCL"]
MAX_CLOUD = 20

STUDY_SITE = "YF"
OUTPUT_DIR = f"E:\planetscope_lake_ice\Data\Input\Machine Learning - Sentinel Samples for RF\{STUDY_SITE}"

AOI_POLYGON = { 
    "type": "Polygon", 
    "coordinates": [[ 
        [-146.332784, 66.458245], 
        [-146.343045, 66.906587], 
        [-145.201146, 66.906587], 
        [-145.211407, 66.458245], 
        [-146.332784, 66.458245], 
    ]] 
}

SEASONS = {
    "Breakup": ("{year}-04-15T00:00:00Z", "{year}-07-15T23:59:59Z"),
    "Freezeup": ("{year}-10-01T00:00:00Z", "{year}-11-30T23:59:59Z")
}

YEARS = [2019,2021,2022,2023,2024,2025]

LAKE_SHP = r"E:\planetscope_lake_ice\Data\Input\Study Sites - Manual ALPOD Data\YF 50x50 km\YF Lakes from ALPOD Shapefile\YF_50x50km_lakes.shp"

In [55]:
import pandas as pd

os.makedirs(OUTPUT_DIR, exist_ok=True)
all_meta = []
for season, (start_tpl, end_tpl) in SEASONS.items():
    for year in YEARS:
        start = start_tpl.format(year=year)
        end = end_tpl.format(year=year)
        items = search_s2_l2a(AOI_POLYGON, start, end, cloud_cover=MAX_CLOUD)
        if not items:
            continue
        df = summarize_metadata(items)
        df["season"] = season
        df["year"] = year
        all_meta.append(df)
        for item in items:
            download_all_bands(item, OUTPUT_DIR, BANDS, SCL_BAND)
            scene_dir = os.path.join(OUTPUT_DIR, item.id)
            combined_tif = os.path.join(scene_dir, f"{item.id}_allbands.tif")
            merge_bands_to_multitif(scene_dir, combined_tif, BANDS, SCL_BAND)
if all_meta:
    metadata_table = pd.concat(all_meta).sort_values("datetime")
    display(metadata_table)

,id,datetime,cloud_cover,sun_azimuth,sun_elevation,asset_count,season,year
81,S2B_6WWV_20190420_0_L2A,2019-04-20T21:34:43.024000Z,16.532818,175.530615,34.474447,35,Breakup,2019
80,S2B_6WWV_20190426_1_L2A,2019-04-26T21:46:42.954000Z,5.707902,181.822125,36.514551,35,Breakup,2019
79,S2B_6WWV_20190426_0_L2A,2019-04-26T21:46:42.956000Z,1.089252,181.821358,36.514512,35,Breakup,2019
78,S2B_6WWU_20190426_1_L2A,2019-04-26T21:46:56.772000Z,0.737660,181.858653,37.410501,35,Breakup,2019
77,S2B_6WWU_20190426_0_L2A,2019-04-26T21:46:56.776000Z,1.714473,181.857877,37.410463,35,Breakup,2019
...,...,...,...,...,...,...,...,...
0,S2B_6WWU_20250713_0_L2A,2025-07-13T21:46:53.191000Z,15.926582,179.316589,45.457593,38,Breakup,2025
3,S2B_6WWV_20251011_0_L2A,2025-10-11T21:46:34.646000Z,0.057509,184.400962,15.437816,38,Freezeup,2025
2,S2B_6WWU_20251011_0_L2A,2025-10-11T21:46:48.469000Z,0.195686,184.433236,16.331230,38,Freezeup,2025
1,S2C_6WWV_20251016_0_L2A,2025-10-16T21:46:46.216000Z,15.238674,184.683372,13.573884,38,Freezeup,2025


# Turn a folder into RGB JPEGs for classification
# ~ --------------------------------- ~ #

In [ ]:
import os
import glob
import rasterio
import numpy as np
from PIL import Image

base_dir = r"E:\planetscope_lake_ice\Data\Input\Machine Learning - Sentinel Samples for RF"
site = "YF"
site_dir = os.path.join(base_dir, site)

output_dir = os.path.join(site_dir, "RGB_JPEGs")
os.makedirs(output_dir, exist_ok=True)

def stretch(band, lower_percent=2, upper_percent=98):
    """Percentile stretch for enhancing contrast."""
    low = np.percentile(band, lower_percent)
    high = np.percentile(band, upper_percent)
    band = np.clip((band - low) / (high - low), 0, 1)
    return (band * 255).astype(np.uint8)

scene_folders = [f for f in glob.glob(os.path.join(site_dir, "S2*")) if os.path.isdir(f)]

for scene_path in scene_folders:
    name = os.path.basename(scene_path)
    print(f"Processing {name}...")

    red_path = os.path.join(scene_path, "B04.tif")
    green_path = os.path.join(scene_path, "B03.tif")
    blue_path = os.path.join(scene_path, "B02.tif")
    

    with rasterio.open(red_path) as r, \
         rasterio.open(green_path) as g, \
         rasterio.open(blue_path) as b:
        red = r.read(1).astype(float)
        green = g.read(1).astype(float)
        blue = b.read(1).astype(float)

    # Stretch each band
    red_s = stretch(red)
    green_s = stretch(green)
    blue_s = stretch(blue)
    
    rgb = np.dstack((red_s, green_s, blue_s))
    img = Image.fromarray(rgb)
    
    out_path = os.path.join(output_dir, f"{name}_RGB.jpg")
    img.save(out_path, "JPEG", quality=90)
    print(f"   Saved {out_path}")